In [31]:
import os
from dotenv import load_dotenv
import openai
import requests
import json
import pandas as pd

In [32]:
load_dotenv()
CB_API_KEY = os.getenv('CB_API_KEY')
openai.api_key_path = None
openai.api_key = os.getenv("OPENAI_API_KEY")

In [33]:
fake_news_articles = pd.read_csv("data/fake-news/train.csv")
fake_news_articles

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [67]:
article_content = """World War II or the Second World War, often abbreviated as WWII or WW2, was a global conflict that lasted from 1939 to 1945. The vast majority of the world's countries, including all of the great powers, fought as part of two opposing military alliances: the Allies and the Axis. Many participants threw their economic, industrial, and scientific capabilities behind this total war, blurring the distinction between civilian and military resources. Aircraft played a major role, enabling the strategic bombing of population centres and the delivery of the only two nuclear weapons ever used in war.

World War II was by far the deadliest conflict in history, resulting in an estimated 70 to 85 million fatalities, mostly among civilians. Tens of millions died due to genocides (including the Holocaust), starvation, massacres, and disease. In the wake of the Axis defeat, Germany and Japan were occupied, and war crimes tribunals were conducted against German and Japanese leaders.

The causes of World War II are debated, but contributing factors included the Second Italo-Ethiopian War, Spanish Civil War, Second Sino-Japanese War, Soviet–Japanese border conflicts, the rise of fascism in Europe, and European tensions in the aftermath of World War I. World War II is generally considered to have begun on 1 September 1939, when Nazi Germany, under Adolf Hitler, invaded Poland. The United Kingdom and France subsequently declared war on Germany on 3 September. Under the Molotov–Ribbentrop Pact of August 1939, Germany and the Soviet Union had partitioned Poland and marked out their "spheres of influence" across Finland, Estonia, Latvia, Lithuania and Romania. From late 1939 to early 1941, in a series of campaigns and treaties, Germany conquered or controlled much of continental Europe, in a military alliance with Italy, Japan and other countries called the Axis. Following the onset of campaigns in North Africa and East Africa, and the fall of France in mid-1940, the war continued primarily between the European Axis powers and the British Empire, with war in the Balkans, the aerial Battle of Britain, the Blitz of the United Kingdom, and the Battle of the Atlantic. On 22 June 1941, Germany led the European Axis powers in an invasion of the Soviet Union, opening the Eastern Front, the largest land theatre of war in history."""

In [35]:
article_content = fake_news_articles.iloc[0]["text"]

In [36]:
article_content

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [50]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text, usually an article. You will always return a numbered list and nothing else. Each item in the numbered list will always start with the topic of the article."},
        {"role": "user", "content": "Give me a numbered list of the most important claims and facts relevant to the argument of the following article. Make sure that each item in the list is a brief, verifiable sentence that contains the claim or fact and any context from the article needed to fact check it. Also give me the argument of the article in as few words as possible. An example of the output is: Argument: Red is the best color.\n1.Red is the coolest\n2.Everyone likes the color red."},
        {"role": "user", "content": article_content}
    ]
)

In [51]:
response["choices"][0]["message"]["content"]

'Argument: House Democrats did not know about Comey’s letter until Republicans tweeted it out.\n1. House Democrats on relevant committees did not receive Comey’s letter about reopening the investigation into Clinton’s emails. \n2. Jason Chaffetz tweeted about the discovery of the emails before the letter was sent to the Democratic ranking members on the committees. \n3. Comey may have notified only Republican committee members in advance, but there is no evidence of this. \n4. Chaffetz’s behavior was irresponsible and reveals a larger problem with House Republicans.\n5. Chaffetz is unlikely to face consequences for this behavior due to his strong Republican district and support from House leadership.'

In [38]:
response_content = response["choices"][0]["message"]["content"]
topic = response_content[response_content.find("Argument: ") + 10:response_content.find("\n")]
facts = response_content[response_content.find("\n") + 2:]
facts = [fact[fact.find(" ") + 1:] for fact in facts.split("\n")]
topic, facts

("FBI Director James Comey sent out a letter announcing that the FBI was reopening its investigation into Hillary Clinton's email server, which was inadvertently made public via a tweet from Oversight Committee Chairman Jason Chaffetz. It has been revealed that the Democratic Ranking Members on relevant committees had not received the letter until after Chaffetz tweeted it, therefore they found out about the news on Twitter. This has caused much controversy and accusations of partisanship.",
 ["The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.",
  'The Oversight Committee Chairman, Jason Chaffetz, did not notify his Democratic counterparts about the FBI review of the emails that had been discovered.',
  'Chaffetz is acting in a partisan way that violates basic standards of fairness by not notifying ranking member Elijah Cummings

In [39]:
response

<OpenAIObject chat.completion id=chatcmpl-76QXlp59E7ca7y2re0r9eWIuY5iVD at 0x7f7b910afb48> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Argument: FBI Director James Comey sent out a letter announcing that the FBI was reopening its investigation into Hillary Clinton's email server, which was inadvertently made public via a tweet from Oversight Committee Chairman Jason Chaffetz. It has been revealed that the Democratic Ranking Members on relevant committees had not received the letter until after Chaffetz tweeted it, therefore they found out about the news on Twitter. This has caused much controversy and accusations of partisanship.\n\n1. The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.\n2. The Oversight Committee Chairman, Jason Chaffetz, did not notify his 

In [40]:
api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/score/text/{topic + '; ' + facts[0]}"
request_headers = {"x-api-key": CB_API_KEY}

api_response = requests.get(url=api_endpoint, headers=request_headers)

print(api_response.json())

{'version': '2', 'claim': "FBI Director James Comey sent out a letter announcing that the FBI was reopening its investigation into Hillary Clinton's email server, which was inadvertently made public via a tweet from Oversight Committee Chairman Jason Chaffetz. It has been revealed that the Democratic Ranking Members on relevant committees had not received the letter until after Chaffetz tweeted it, therefore they found out about the news on Twitter. This has caused much controversy and accusations of partisanship.; The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.", 'results': [{'text': "FBI Director James Comey sent out a letter announcing that the FBI was reopening its investigation into Hillary Clinton's email server, which was inadvertently made public via a tweet from Oversight Committee Chairman Jason Chaffetz. It has been

In [41]:
api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/knowledge_bases/{facts[0]}"
request_headers = {"x-api-key": CB_API_KEY}

api_response = requests.get(url=api_endpoint, headers=request_headers)

print(api_response.json())

{'claim': "The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.", 'origin': 'Claim Checker - Knowledge Bases', 'justification': []}


In [42]:
api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/fact_matcher/{facts[0]}"
request_headers = {"x-api-key": CB_API_KEY}

api_response = requests.get(url=api_endpoint, headers=request_headers)

print(api_response.json())

{'claim': "The Democratic Ranking Members on relevant committees had not received Comey's letter about reopening the investigation into Clinton's email server until after Jason Chaffetz tweeted it out and made it public.", 'origin': 'Claim Matcher', 'justification': []}


In [82]:
def score_facts_for_validation(facts):
    api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/score/text/sentences/{' '.join(facts)}"
    request_headers = {"x-api-key": CB_API_KEY}

    api_response = requests.get(url=api_endpoint, headers=request_headers)

    return api_response.json()['results']

def get_justification(fact):
    api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/knowledge_bases/{fact}"
    request_headers = {"x-api-key": CB_API_KEY}

    api_response = requests.get(url=api_endpoint, headers=request_headers)

    return api_response.json()['justification']

def get_matching_facts(fact):
    api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/fact_matcher/{fact}"
    request_headers = {"x-api-key": CB_API_KEY}

    api_response = requests.get(url=api_endpoint, headers=request_headers)

    return api_response.json()['justification']
    
def get_topic_and_claims(article_text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text, usually an article. You will always return a numbered list and nothing else. Each item in the numbered list will always start with the topic of the article."},
            {"role": "user", "content": "Give me a numbered list of the most important claims and facts relevant to the argument of the following article. Make sure that each item in the list is a brief, verifiable sentence that contains the claim or fact and any context from the article needed to fact check it. Also give me the argument of the article in as few words as possible. An example of the output is: Argument: World War II was a major global conflict.\n1. World War II began in 1939\n2. 27 million people died during World War II"},
            {"role": "user", "content": article_content}
        ]
    )
    
    response_content = response["choices"][0]["message"]["content"]
    topic = response_content[response_content.find("Argument: ") + 10:response_content.find("\n")]
    facts = response_content[response_content.find("\n") + 2:]
    facts = [fact[fact.find(" ") + 1:] for fact in facts.split("\n")]
    return topic, facts

def validate_facts(topic, facts):
    facts = [topic] + facts
    scores = score_facts_for_validation(facts)
    
    print(f"Topic: {topic}")
    for i in range(len(facts)):
        print("------------------------")
        print(f"Fact: {facts[i]}")
        print(f"Score: {scores[i]['score']}")
        if scores[i]['score'] > 0.5:
            sentence = facts[i]
            justification = get_justification(sentence)
            print(f"Justification: {justification}")
            matches = get_matching_facts(sentence)
            print(f"Matches: {matches}")

In [68]:
topic, facts = get_topic_and_claims(article_content)

In [ ]:
validate_facts(topic, facts)